In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn import preprocessing

In [2]:
df = pd.read_csv(r'C:\Users\aarti\Downloads\churn_modelling.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)
df.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [5]:
#Distinguish the feature and target set and divide the data set into training and test sets.
X=df.iloc[:, :df.shape[1]-1].values       #Independent Variables
y=df.iloc[:, -1].values                   #Dependent Variable
X.shape, y.shape

((10000, 10), (10000,))

In [6]:
print(X[:8,1], '... will now become: ')

label_X_country_encoder = LabelEncoder()
X[:,1] = label_X_country_encoder.fit_transform(X[:,1])
print(X[:8,1])

['France' 'Spain' 'France' 'France' 'Spain' 'Spain' 'France' 'Germany'] ... will now become: 
[0 2 0 0 2 2 0 1]


In [7]:
print(X[:6,2], '... will now become: ')

label_X_gender_encoder = LabelEncoder()
X[:,2] = label_X_gender_encoder.fit_transform(X[:,2])
print(X[:6,2])

['Female' 'Female' 'Female' 'Female' 'Female' 'Male'] ... will now become: 
[0 0 0 0 0 1]


In [8]:
#Split the countries into respective dimensions. Converting the string features into their own dimensions.

transform = ColumnTransformer([("countries", OneHotEncoder(), [1])], remainder="passthrough") # 1 is the country column
X = transform.fit_transform(X)
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [9]:
#Dimensionality reduction. A 0 on two countries means that the country has to be the one variable which wasn't included

X = X[:,1:]
X.shape

(10000, 11)

In [10]:
#Splitting the Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
#Normalize the train and test data

['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']

sc=StandardScaler()
X_train[:,np.array([2,4,5,6,7,10])] = sc.fit_transform(X_train[:,np.array([2,4,5,6,7,10])])
X_test[:,np.array([2,4,5,6,7,10])] = sc.transform(X_test[:,np.array([2,4,5,6,7,10])])

In [12]:
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train

array([[-0.5698444 ,  1.74309049,  0.16958176, ...,  0.64259497,
        -1.03227043,  1.10643166],
       [ 1.75486502, -0.57369368, -2.30455945, ...,  0.64259497,
         0.9687384 , -0.74866447],
       [-0.5698444 , -0.57369368, -1.19119591, ...,  0.64259497,
        -1.03227043,  1.48533467],
       ...,
       [-0.5698444 , -0.57369368,  0.9015152 , ...,  0.64259497,
        -1.03227043,  1.41231994],
       [-0.5698444 ,  1.74309049, -0.62420521, ...,  0.64259497,
         0.9687384 ,  0.84432121],
       [ 1.75486502, -0.57369368, -0.28401079, ...,  0.64259497,
        -1.03227043,  0.32472465]])

In [13]:
from tensorflow.keras.models import Sequential     #Initialize & build the model
classifier = Sequential()                          # Initializing the ANN

In [14]:
from tensorflow.keras.layers import Dense
classifier.add(Dense(activation = 'relu', input_dim = 11, units=256, kernel_initializer='uniform'))

In [15]:
# Adding the hidden layer
classifier.add(Dense(activation = 'relu', units=512, kernel_initializer='uniform'))
classifier.add(Dense(activation = 'relu', units=256, kernel_initializer='uniform'))
classifier.add(Dense(activation = 'relu', units=128, kernel_initializer='uniform'))

In [16]:
classifier.add(Dense(activation = 'sigmoid', units=1, kernel_initializer='uniform'))   #o/p layer

In [17]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # Compile the model

In [18]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               3072      
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dense_3 (Dense)             (None, 128)               32896     
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 299,009
Trainable params: 299,009
Non-trainable params: 0
_________________________________________________________________


In [19]:
classifier.fit(
    X_train, y_train,           
    validation_data=(X_test,y_test),
    epochs=20,
    batch_size=32
)

Epoch 1/20
250/250 [==============================] - 3s 5ms/step - loss: 0.4264 - accuracy: 0.8177 - val_loss: 0.3737 - val_accuracy: 0.8535
Epoch 2/20
250/250 [==============================] - 1s 4ms/step - loss: 0.3607 - accuracy: 0.8510 - val_loss: 0.3524 - val_accuracy: 0.8565
Epoch 3/20
250/250 [==============================] - 1s 3ms/step - loss: 0.3494 - accuracy: 0.8570 - val_loss: 0.3547 - val_accuracy: 0.8575
Epoch 4/20
250/250 [==============================] - 1s 3ms/step - loss: 0.3443 - accuracy: 0.8614 - val_loss: 0.3415 - val_accuracy: 0.8570
Epoch 5/20
250/250 [==============================] - 1s 3ms/step - loss: 0.3394 - accuracy: 0.8637 - val_loss: 0.3373 - val_accuracy: 0.8610
Epoch 6/20
250/250 [==============================] - 1s 3ms/step - loss: 0.3350 - accuracy: 0.8645 - val_loss: 0.3354 - val_accuracy: 0.8560
Epoch 7/20
250/250 [==============================] - 1s 3ms/step - loss: 0.3308 - accuracy: 0.8668 - val_loss: 0.3466 - val_accuracy: 0.8610
Epoch 

In [20]:
#Predict the results using 0.5 as a threshold
y_pred = classifier.predict(X_test)
y_pred

array([[0.3689418 ],
       [0.15950903],
       [0.14942706],
       ...,
       [0.03482229],
       [0.10991234],
       [0.22517693]], dtype=float32)

In [21]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [22]:
#Print the Accuracy score and confusion matrix
from sklearn.metrics import confusion_matrix,classification_report
cm1 = confusion_matrix(y_test, y_pred)
cm1

array([[1515,   80],
       [ 206,  199]], dtype=int64)

In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.95      0.91      1595
           1       0.71      0.49      0.58       405

    accuracy                           0.86      2000
   macro avg       0.80      0.72      0.75      2000
weighted avg       0.85      0.86      0.85      2000



In [24]:
accuracy_model1 = ((cm1[0][0]+cm1[1][1])*100)/(cm1[0][0]+cm1[1][1]+cm1[0][1]+cm1[1][0])
print (accuracy_model1, '% of testing data was classified correctly')

85.7 % of testing data was classified correctly
